In [1]:
import pandas as pd
import numpy as np
import json

## Загружаем данные

In [2]:
df = pd.read_csv('data/Nobel_Laureates/archive.csv')

In [3]:
# Для контроля количества строк
df.shape

(969, 18)

In [4]:
df_country = pd.read_csv('data/Countries_of_the_World/countries_of_the_world.csv')

In [5]:
df_city = pd.read_csv('data/World_Cities/worldcitiespop.csv')

/tmp/ipykernel_2655/1389754110.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_city = pd.read_csv('data/World_Cities/worldcitiespop.csv')


In [6]:
with open('data/Country_Data/continent.json') as f:
    continent = json.load(f)

df_continent = pd.DataFrame([continent.keys(), continent.values()]).T.rename(columns={0:'region', 1: 'continent'})    

In [7]:
with open('data/Country_Data/currency.json') as f:
    currency = json.load(f) 
    
# Валюта
df_currency = pd.DataFrame([currency.keys(), currency.values()]).T.rename(columns={0:'region', 1: 'currency'})

In [8]:
with open('data/Country_Data/iso3.json') as f:
    iso3 = json.load(f) 
    
# Регион 3 символа
df_iso3 = pd.DataFrame([iso3.keys(), iso3.values()]).T.rename(columns={0:'region', 1: 'reg_3ch'})

In [9]:
with open('data/Country_Data/names.json') as f:
    names = json.load(f) 
    
# Название региона
df_names = pd.DataFrame([names.keys(), names.values()]).T.rename(columns={0:'region', 1: 'reg_name'})

In [10]:
with open('data/Country_Data/phone.json') as f:
    phone = json.load(f) 
    
# Телефонныей код региона
df_phone = pd.DataFrame([phone.keys(), phone.values()]).T.rename(columns={0:'region', 1: 'phone_cod'})

In [11]:
df_region = df_iso3.merge(df_names, how='left', on='region')\
                   .merge(df_phone, how='left', on='region')\
                   .merge(df_currency, how='left', on='region')\
                   .merge(df_continent, how='left', on='region')

## Приведение названий к единому 

In [12]:
df_country['Country'] = df_country['Country'].apply(lambda s: s.strip())
df_country['Country'] = df_country['Country'].apply(lambda s: s.replace('&', 'and'))

In [13]:
list_co = ['Bahamas, The', 'British Virgin Is.', 'Central African Rep.', 'Congo, Dem. Rep.',
          'Congo, Repub. of the', "Cote d'Ivoire", 'Gambia, The', 'Korea, North', 'Korea, South', 'Macau',
          'Micronesia, Fed. St.', 'N. Mariana Islands', 'St Pierre and Miquelon', 'Turks and Caicos Is',
          'Virgin Islands',]
          
list_re = ['Bahamas', 'British Virgin Islands', 'Central African Republic', 'Democratic Republic of the Congo',
          'Republic of the Congo', 'Ivory Coast', 'Gambia', 'North Korea', 'South Korea', 'Macao', 
          'Micronesia', 'Northern Mariana Islands', 'Saint Pierre and Miquelon', 'Turks and Caicos Islands',
          'U.S. Virgin Islands']

dict_country = {list_co[i]: list_re[i] for i in range(len(list_co))}
dict_country_df = {'Northern Ireland':'Northern Mariana Islands',
               'Guadeloupe Island':'Guadeloupe',
               'Republic of Macedonia': 'Macedonia',
               'Trinidad':'Trinidad and Tobago',
               'United States of America':'United States'}

In [14]:
df_country['Country'] = df_country['Country'].apply(lambda c: dict_country[c] if c in list_co else c )

In [15]:
# создадим отдельный столбец в основном датасете, за основную страну возьмем страну рождения и 
# и в случае отсутвия информации дополним страной организации

def join_country(row):
    if type(row['Birth Country']) == str:
        return row['Birth Country']
    else:    
        return row['Organization Country']

df['Country'] = df.apply(join_country, axis=1)

In [16]:
# Приведем названия стран к общему виду.

def processing_country(country):
    if type(country) == str and '(' in country:
        ind_s = country.find('(')+1
        ind_f = country.find(')')
        country = country[ind_s:ind_f] 
    
    return country

df['Country'] = df['Country'].apply(processing_country)

In [17]:
df['Country'] = df['Country'].apply(lambda c: dict_country_df[c] if c in dict_country_df.keys()  else c )

In [18]:
df.shape

(969, 19)

In [19]:
# # создадим отдельный столбец в основном датасете, за основную страну возьмем город рождения и 
# # и в случае отсутвия информации дополним городом организации

# def join_city(row):
#     if type(row['Birth City']) == str:
#         return row['Birth City']
#     else:    
#         return row['Organization City']

# df['City'] = df.apply(join_city, axis=1)

In [20]:
# df['City'] = df['City'].apply(lambda s: s.lower() if type(s) == str else s)

In [21]:
# df_city = df_city.rename(columns={'Region': 'RegionCity'})

In [22]:
# Удалим полные дубликаты строк
df_city = df_city.drop_duplicates()

In [23]:
# Проверим на дублированность городов
df_city['City'].value_counts()

san jose             328
san antonio          320
santa rosa           296
santa cruz           282
san juan             255
                    ... 
magyarszentmiklos      1
magyarszek             1
magyarszecsod          1
magyarsarlos           1
zvishavane             1
Name: City, Length: 2351891, dtype: int64

In [24]:
# Получается у нас есть несколько одинаковых названий городов, 
# но с разными регионами внутри, координатами и находящиеся в разных странах
# Просто смержить не поучиться, даже если учтем страну и город,
# то как выбрать регион? Ценная информаця тут -координаты. 
# Пока оставим просто эту таблицу отдельно 

df_city[df_city['City']=='san jose']

,Country,City,AccentCity,Region,Population,Latitude,Longitude
107723,an,san jose,San José,0.0,NaN,12.116667,-68.233333
134621,ar,san jose,San Jose,01,NaN,-37.507633,-61.918198
134622,ar,san jose,San Jose,02,NaN,-26.790873,-66.070697
134623,ar,san jose,San Jose,05,NaN,-30.007866,-64.623982
134624,ar,san jose,San Jose,13,NaN,-32.364626,-68.255877
...,...,...,...,...,...,...,...
3075160,ve,san jose,San José,20,NaN,7.790556,-72.223056
3075161,ve,san jose,San José,21,NaN,9.409444,-70.451667
3075162,ve,san jose,San Jose,22,NaN,10.334722,-68.663056
3075163,ve,san jose,San José,23,NaN,10.016667,-72.383333


## Слияние таблиц

In [25]:
df_country = df_region.merge(df_country, how='left', left_on='reg_name', right_on='Country')

In [26]:
sum(df_country['reg_name'] == df_country['Country'])

223

In [27]:
df_country['Country'].isna().sum()

27

In [28]:
df_country = df_country.drop(['Country', 'Region'], axis=1)

In [29]:
df = df.merge(df_country, how='left', left_on='Country', right_on='reg_name')

In [30]:
df.shape

(969, 43)

In [31]:
df

,Year,Category,Prize,Motivation,Prize Share,Laureate ID,Laureate Type,Full Name,Birth Date,Birth City,...,Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,1901,Chemistry,The Nobel Prize in Chemistry 1901,"""in recognition of the extraordinary services ...",1/1,160,Individual,Jacobus Henricus van 't Hoff,1852-08-30,Rotterdam,...,"460,8","26,71","0,97","72,32",3,"10,9","8,68","0,021","0,244","0,736"
1,1901,Literature,The Nobel Prize in Literature 1901,"""in special recognition of his poetic composit...",1/1,569,Individual,Sully Prudhomme,1839-03-16,Paris,...,"586,4","33,53","2,07","64,4",4,"11,99","9,14","0,022","0,214","0,764"
2,1901,Medicine,The Nobel Prize in Physiology or Medicine 1901,"""for his work on serum therapy, especially its...",1/1,293,Individual,Emil Adolf von Behring,1854-03-15,Hansdorf (Lawice),...,"306,3","45,91","1,12","52,97",3,"9,85","9,89","0,05","0,311","0,64"
3,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,462,Individual,Jean Henry Dunant,1828-05-08,Geneva,...,"680,9","10,42","0,61","88,97",3,"9,71","8,49","0,015","0,34","0,645"
4,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,463,Individual,Frédéric Passy,1822-05-20,Paris,...,"586,4","33,53","2,07","64,4",4,"11,99","9,14","0,022","0,214","0,764"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,2016,Medicine,The Nobel Prize in Physiology or Medicine 2016,"""for his discoveries of mechanisms for autophagy""",1/1,927,Individual,Yoshinori Ohsumi,1945-02-09,Fukuoka,...,"461,2","12,19","0,96","86,85",3,"9,37","9,16","0,017","0,258","0,725"
965,2016,Peace,The Nobel Peace Prize 2016,"""for his resolute efforts to bring the country...",1/1,934,Individual,Juan Manuel Santos,1951-08-10,Bogotá,...,"176,2","2,42","1,67","95,91",2,"20,48","5,58","0,125","0,342","0,533"
966,2016,Physics,The Nobel Prize in Physics 2016,"""for theoretical discoveries of topological ph...",1/2,928,Individual,David J. Thouless,1934-09-21,Bearsden,...,"543,5","23,46","0,21","76,33",3,"10,71","10,13","0,005","0,237","0,758"
967,2016,Physics,The Nobel Prize in Physics 2016,"""for theoretical discoveries of topological ph...",1/4,929,Individual,F. Duncan M. Haldane,1951-09-14,London,...,"543,5","23,46","0,21","76,33",3,"10,71","10,13","0,005","0,237","0,758"


In [32]:
df.isna().sum()

Year                                    0
Category                                0
Prize                                   0
Motivation                             88
Prize Share                             0
Laureate ID                             0
Laureate Type                           0
Full Name                               0
Birth Date                             29
Birth City                             28
Birth Country                          26
Sex                                    26
Organization Name                     247
Organization City                     253
Organization Country                  253
Death Date                            352
Death City                            370
Death Country                         364
Country                                26
region                                 36
reg_3ch                                36
reg_name                               36
phone_cod                              36
currency                          

In [33]:
# Переименование колонок всех в нижний регистр

df = df.rename(columns = str.lower)

In [34]:
df = df.drop(['reg_name'], axis=1)

Так как мы не знаем, что конкретно будем анализировать, какие гипотизы проверять и какие данные нам могут понадобиться, то оставим пока все колонки. Удалить всегда можно.

Выгрузим только итоговую таблицу, которую мы тут подготовили, обработав информацию по странам. Данные погородам только глянули, но ничего с ними не делали.

In [35]:
df.to_csv('data/df.csv', index=False, sep=';')

In [36]:
# проверка выгрузки
pd.read_csv('data/df.csv', sep=';')

,year,category,prize,motivation,prize share,laureate id,laureate type,full name,birth date,birth city,...,phones (per 1000),arable (%),crops (%),other (%),climate,birthrate,deathrate,agriculture,industry,service
0,1901,Chemistry,The Nobel Prize in Chemistry 1901,"""in recognition of the extraordinary services ...",1/1,160,Individual,Jacobus Henricus van 't Hoff,1852-08-30,Rotterdam,...,"460,8","26,71","0,97","72,32",3,"10,9","8,68","0,021","0,244","0,736"
1,1901,Literature,The Nobel Prize in Literature 1901,"""in special recognition of his poetic composit...",1/1,569,Individual,Sully Prudhomme,1839-03-16,Paris,...,"586,4","33,53","2,07","64,4",4,"11,99","9,14","0,022","0,214","0,764"
2,1901,Medicine,The Nobel Prize in Physiology or Medicine 1901,"""for his work on serum therapy, especially its...",1/1,293,Individual,Emil Adolf von Behring,1854-03-15,Hansdorf (Lawice),...,"306,3","45,91","1,12","52,97",3,"9,85","9,89","0,05","0,311","0,64"
3,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,462,Individual,Jean Henry Dunant,1828-05-08,Geneva,...,"680,9","10,42","0,61","88,97",3,"9,71","8,49","0,015","0,34","0,645"
4,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,463,Individual,Frédéric Passy,1822-05-20,Paris,...,"586,4","33,53","2,07","64,4",4,"11,99","9,14","0,022","0,214","0,764"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,2016,Medicine,The Nobel Prize in Physiology or Medicine 2016,"""for his discoveries of mechanisms for autophagy""",1/1,927,Individual,Yoshinori Ohsumi,1945-02-09,Fukuoka,...,"461,2","12,19","0,96","86,85",3,"9,37","9,16","0,017","0,258","0,725"
965,2016,Peace,The Nobel Peace Prize 2016,"""for his resolute efforts to bring the country...",1/1,934,Individual,Juan Manuel Santos,1951-08-10,Bogotá,...,"176,2","2,42","1,67","95,91",2,"20,48","5,58","0,125","0,342","0,533"
966,2016,Physics,The Nobel Prize in Physics 2016,"""for theoretical discoveries of topological ph...",1/2,928,Individual,David J. Thouless,1934-09-21,Bearsden,...,"543,5","23,46","0,21","76,33",3,"10,71","10,13","0,005","0,237","0,758"
967,2016,Physics,The Nobel Prize in Physics 2016,"""for theoretical discoveries of topological ph...",1/4,929,Individual,F. Duncan M. Haldane,1951-09-14,London,...,"543,5","23,46","0,21","76,33",3,"10,71","10,13","0,005","0,237","0,758"


In [37]:
df.columns

Index(['year', 'category', 'prize', 'motivation', 'prize share', 'laureate id',
       'laureate type', 'full name', 'birth date', 'birth city',
       'birth country', 'sex', 'organization name', 'organization city',
       'organization country', 'death date', 'death city', 'death country',
       'country', 'region', 'reg_3ch', 'phone_cod', 'currency', 'continent',
       'population', 'area (sq. mi.)', 'pop. density (per sq. mi.)',
       'coastline (coast/area ratio)', 'net migration',
       'infant mortality (per 1000 births)', 'gdp ($ per capita)',
       'literacy (%)', 'phones (per 1000)', 'arable (%)', 'crops (%)',
       'other (%)', 'climate', 'birthrate', 'deathrate', 'agriculture',
       'industry', 'service'],
      dtype='object')

In [38]:
df[[ 'country','region', 'reg_3ch', 'phone_cod', 'currency',
       'continent', 'population', 'area (sq. mi.)',
       'pop. density (per sq. mi.)', 'coastline (coast/area ratio)',
       'net migration', 'infant mortality (per 1000 births)',
       'gdp ($ per capita)', 'literacy (%)', 'phones (per 1000)', 'arable (%)',
       'crops (%)', 'other (%)', 'climate', 'birthrate', 'deathrate',
       'agriculture', 'industry', 'service']]

,country,region,reg_3ch,phone_cod,currency,continent,population,area (sq. mi.),pop. density (per sq. mi.),coastline (coast/area ratio),...,phones (per 1000),arable (%),crops (%),other (%),climate,birthrate,deathrate,agriculture,industry,service
0,Netherlands,NL,NLD,31,EUR,EU,16491461.0,41526.0,"397,1","1,09",...,"460,8","26,71","0,97","72,32",3,"10,9","8,68","0,021","0,244","0,736"
1,France,FR,FRA,33,EUR,EU,60876136.0,547030.0,"111,3","0,63",...,"586,4","33,53","2,07","64,4",4,"11,99","9,14","0,022","0,214","0,764"
2,Poland,PL,POL,48,PLN,EU,38536869.0,312685.0,"123,3","0,16",...,"306,3","45,91","1,12","52,97",3,"9,85","9,89","0,05","0,311","0,64"
3,Switzerland,CH,CHE,41,CHF,EU,7523934.0,41290.0,"182,2","0,00",...,"680,9","10,42","0,61","88,97",3,"9,71","8,49","0,015","0,34","0,645"
4,France,FR,FRA,33,EUR,EU,60876136.0,547030.0,"111,3","0,63",...,"586,4","33,53","2,07","64,4",4,"11,99","9,14","0,022","0,214","0,764"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,Japan,JP,JPN,81,JPY,AS,127463611.0,377835.0,"337,4","7,87",...,"461,2","12,19","0,96","86,85",3,"9,37","9,16","0,017","0,258","0,725"
965,Colombia,CO,COL,57,COP,SA,43593035.0,1138910.0,"38,3","0,28",...,"176,2","2,42","1,67","95,91",2,"20,48","5,58","0,125","0,342","0,533"
966,United Kingdom,GB,GBR,44,GBP,EU,60609153.0,244820.0,"247,6","5,08",...,"543,5","23,46","0,21","76,33",3,"10,71","10,13","0,005","0,237","0,758"
967,United Kingdom,GB,GBR,44,GBP,EU,60609153.0,244820.0,"247,6","5,08",...,"543,5","23,46","0,21","76,33",3,"10,71","10,13","0,005","0,237","0,758"


In [39]:
'region', 'reg_3ch', 'reg_name', 'phone_cod', 'currency',
       'continent','population', 'area (sq. mi.)',
       'pop. density (per sq. mi.)', 'coastline (coast/area ratio)',
       'net migration', 'infant mortality (per 1000 births)',
       'gdp ($ per capita)', 'literacy (%)', 'phones (per 1000)', 'arable (%)',
       'crops (%)', 'other (%)', 'climate', 'birthrate', 'deathrate',
       'agriculture', 'industry', 'service'

IndentationError: unexpected indent (1290940149.py, line 2)